In [1]:
import aocd
from aocd.models import Puzzle
day = 21
year = 2021
puzzle = Puzzle(year=year, day=day)
# data = aocd.get_data(day=day, year=year)
with open('./data/input_{:02d}'.format(day), 'w') as fh:
    fh.write(puzzle.input_data)

In [2]:
data = puzzle.input_data.splitlines()
len(data), data[:10]

(2, ['Player 1 starting position: 1', 'Player 2 starting position: 10'])

In [5]:
test_data = """Player 1 starting position: 4
Player 2 starting position: 8""".splitlines()

In [42]:
class Die():
    def __init__(self):
        self.numrolls = 0
    
class DeterministicDie(Die):
    def __init__(self):
        super().__init__()
        self.cnt = 0
        self.maxcnt = 100
    
    def __next__(self):
        self.cnt += 1
        if self.cnt > self.maxcnt:
            self.cnt = 1
        self.numrolls += 1
        return self.cnt
    
class DiracDie(Die):
    def __init__(self):
        super().__init__()
        self.cnt = 0
        self.maxcnt = 100    


class Game():
    def __init__(self, pos1, pos2, die):
        self.pos = [pos1, pos2]
        self.maxnum = 10
        self.die = die
        self.score = [0, 0]
        self.cur = 0
        self.numturns = 0
        
    def turn(self):
        rolls = []
        for i in range(3):
            rolls.append(next(self.die))
        total = sum(rolls)
        cur = self.cur
        self.pos[cur] += total
        if self.pos[cur] > self.maxnum:
            self.pos[cur] = ((self.pos[cur] - 1) % self.maxnum) + 1
        self.score[cur] += self.pos[cur]
        self.cur = (self.cur + 1) % 2
        
    def play(self):
        while self.score[0] < 1000 and self.score[1] < 1000:
            self.turn()
            self.numturns += 1
        if self.score[0] >= 1000:
            return 0
        else: 
            return 1

In [43]:
game = Game(4, 8, DeterministicDie())

In [44]:
game.play()

0

In [46]:
game.die.numrolls

993

In [39]:
game.score

[1000, 745]

In [57]:
game = Game(1, 10, DeterministicDie())
winner = game.play()

In [60]:
puzzle.answer_a = game.score[1-winner] * game.die.numrolls

That's the right answer!  You are one gold star closer to finding the sleigh keys. [Continue to Part Two]


In [93]:
# Part B

In [124]:
from itertools import product
from collections import Counter
import numpy as np

In [113]:
# possible sums of 3 dice
a = list(itertools.product(range(1, 4), range(1, 4), range(1, 4)))
npath_roll = Counter([sum(x) for x in a])

In [289]:
maxscore = 21
maxturns = maxscore
maxpos = 10 


In [290]:
dice_possible = np.zeros(maxpos)
for idx, npath in npath_roll.items():
    dice_possible[idx] = npath

In [291]:
dice_matrix = [np.roll(dice_possible, i) for i in range(maxpos)]
dice_matrix = np.array(dice_matrix, dtype=np.int)
# dice_matrix = dice_matrix[:, :, np.newaxis]

In [487]:
%%time
# turn, pos, score
def get_num_paths(init_pos):
    num_paths = np.zeros((maxturns+1, maxpos, maxscore+1), dtype=np.uint64)
    num_paths[0, init_pos-1, 0] = 1
    # num_paths[0, 8, 10] = 2

    for turn in range(maxturns):
        for pos in range(maxpos):
            for score in range(maxscore):
                npath = num_paths[turn, pos, score]
                if npath == 0:
                    continue
                npath_pos = npath * dice_matrix[pos]
    #             print(turn, pos, score, npath_pos)
                for i, x in enumerate(npath_pos):                
                    new_score = score + i + 1
                    if new_score > maxscore:
                        new_score = maxscore

                    num_paths[turn+1, i, new_score] += npath_pos[i]
    #                 print(new_score, npath_pos[i])
                
                
#     if turn > 2:
#         break
    return num_paths

CPU times: user 9 µs, sys: 2 µs, total: 11 µs
Wall time: 55.6 µs


In [492]:
num_paths_A = get_num_paths(1)
num_paths_B = get_num_paths(10)

In [493]:
num_turn_win_A = num_paths_A[:,:,maxscore].sum(axis=1)
num_turn_win_B = num_paths_B[:,:,maxscore].sum(axis=1)

num_turn_A = num_paths_A[:,:,:maxscore].sum(axis=(1, 2))
num_turn_B = num_paths_B[:,:,:maxscore].sum(axis=(1, 2))

In [494]:
num_universes_A = (num_turn_win_A * np.roll(num_turn_B, 1)).sum()
num_universes_B = (num_turn_win_B * np.roll(num_turn_A, 0)).sum()

In [495]:
num_universes_A, num_universes_B

(57328067654557, 32971421421058)

In [486]:
puzzle.answer_b = num_universes_A

That's the right answer!  You are one gold star closer to finding the sleigh keys.You have completed Day 21! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].
